In [1]:
import numpy as np
import plotly.graph_objs as go
from plotly.offline import init_notebook_mode, iplot

In [2]:
init_notebook_mode(connected=True)


In [3]:
def mccormik_function(t):
    x = t[0]
    y = t[1]
    return np.sin(x + y) \
           + np.square(x - y) \
           - 1.5 * x \
           + 2.5 * y \
           + 1


def calc_gradient_mccormik_function(t):
    x = t[0]
    y = t[1]
    return [np.cos(x + y) + 2*(x - y) - 1.5, np.cos(x + y) - 2*(x - y) + 2.5]


In [4]:
def step(v, direction, learning_rate):
    return v - direction * np.asarray(learning_rate)


def is_points_not_close_enough(cur_x, next_x, tolerance):
    a_cur_x = np.asarray(cur_x)
    a_next_x = np.asarray(next_x)
    distance = np.asarray(a_cur_x - a_next_x)
    distance_s = np.abs(np.sum(distance))
    return distance_s > tolerance


def gradient_descent_multiple_variable(func_calc_gradient, func, start_x=(-4, -5), tolerance=0.00001,
                                       learning_rate=(0.01, 0.01)):
    cur_x = start_x
    convergence_path = [cur_x]
    is_not_converged = True

    while is_not_converged:
        cur_gradient = func_calc_gradient(cur_x)
        next_x = step(cur_x, cur_gradient, learning_rate)
        is_not_converged = is_points_not_close_enough(cur_x, next_x, tolerance)
        convergence_path.append(next_x)
        cur_x = next_x
    return cur_x, func(cur_x), convergence_path

In [5]:
t1 = np.arange(-3, 4, 0.5)
t2 = np.arange(-3, 4, 0.5)


In [6]:
x,y = np.meshgrid(t1, t2)
z = mccormik_function([x,y])


In [7]:
s_a  = (-2.5, 3.5)
s_a_v = mccormik_function(s_a)


In [8]:
min_x, min_f, c_s_path = gradient_descent_multiple_variable(func_calc_gradient=calc_gradient_mccormik_function,
                                                     func=mccormik_function, 
                                                     start_x=s_a,
                                                     learning_rate=(0.01, 0.01),     
                                                     tolerance=0.00001)

In [9]:

func_trace = go.Surface(x=x, y=y, z=z, colorscale='Viridis', opacity=0.8)


In [10]:
minimum_trace = go.Scatter3d(
    x=[min_x[0]],
    y=[min_x[1]],
    z=[min_f],  
    marker=dict(
        color='red',
        size=7,
        symbol="circle"
    ),
    name="Minimum",
    text="Minimum"
)


In [11]:
s_a_trace = go.Scatter3d(
    x=[s_a[0]],
    y=[s_a[1]],
    z=[s_a_v],  
    marker=dict(
        color='blue',
        size=7,
        symbol="circle"
    ),
    name="Start approximation",
    text="Start approximation"
)


In [12]:
convergence_path_trace = go.Scatter3d(
    x=np.transpose(c_s_path)[0],
    y=np.transpose(c_s_path)[1],
    z=list(map(mccormik_function, c_s_path)),  
    marker=dict(
        color='yellow',
        size=4,
        symbol="circle"
    ),
    name="Convergence path",
    text="Convergence path"
)


In [13]:
layout = go.Layout(
    legend=dict(x=-.1, y=1.2),
    autosize=False,
    width=1000,
    height=1000,
    margin=go.layout.Margin(
        l=50,
        r=50,
        b=100,
        t=100,
        pad=4
    )
)



In [14]:
fig = go.Figure(data=[func_trace, s_a_trace, minimum_trace, convergence_path_trace], 
                 layout=layout)
iplot(fig)

